In [120]:
import scrapy
import scrapy.crawler as crawler
from scrapy.utils.log import configure_logging
from multiprocessing import Process, Queue
from twisted.internet import reactor
from pydispatch import dispatcher
import logging
import json
import re
import os
import pandas as pd
import numpy as np
import gc
import copy

from datasets import load_dataset
from datasets.arrow_dataset import Dataset

from bs4 import BeautifulSoup
from lyricsgenius import Genius
import pickle

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('nlp')
logger.setLevel(logging.INFO)

[nltk_data] Downloading package punkt to /home/masdevas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dataset = load_dataset("yelp_review_full")

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset['train']

Dataset({
    features: ['label', 'text'],
    num_rows: 650000
})

In [4]:
df1 = pd.DataFrame({"a": [1, 2, 3]})
df1_ = Dataset.from_pandas(df1)

In [5]:
df1_

Dataset({
    features: ['a'],
    num_rows: 3
})

# Fetch sentences

In [6]:
def get_web_data(function):
    def f(q):
        try:
            json_data = function()
            q.put(json.dumps(json_data))
        except Exception as e:
            q.put(e)
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()
    gc.collect()
    try:
        json_data = json.loads(result)
    except:
        raise result
    return json_data

## USA government RSS channels parsing

In [7]:
def get_usagov_rss_data():
    storage = {}
    try:
        class UsaGovSpider(scrapy.Spider):
            name = "usagov_checker"
            start_urls = ['https://www.state.gov/rss-feeds/']
            
            def parse(self, response, depth=0):
                if depth == 0:
                    a_selectors = response.xpath("//a")
                    for selector in a_selectors:
                        link = selector.xpath("@href").extract_first()
                        if 'rss' in link:
                            yield response.follow(link, self.parse, cb_kwargs={'depth' : 1})
                else:
                    storage[str(response.url)] = response.body.decode("utf-8")

        runner = crawler.CrawlerRunner()
        deferred = runner.crawl(UsaGovSpider)
        deferred.addBoth(lambda _: reactor.stop())
        reactor.run()
        return storage
    except Exception as e:
        return {'exception' : str(e)}

## Joke RSS channels parsing

In [8]:
def get_joke_rss_data():
    storage = {}
    try:
        class JokeSpider(scrapy.Spider):
            name = "joke_checker"
            #start_urls = ['https://blog.feedspot.com/jokes_rss_feeds/'] # 403 error - bot detected, access denied
            start_urls = [
                'http://www.jokesoftheday.net/jokes-feed/', 
                'http://www.funnyshortjokes.com/feed',
                'https://laffgaff.com/feed/',
                'https://www.keeplaughingforever.com/blog//blog-feed.xml',
                'https://lite92.ca/category/joke-of-the-day/feed/',
                'https://www.thelaughline.com/feed/',
                'https://www.jokesbykids.com/riddle/feed/',
                'https://newbloggycat.com/category/good-clean-jokes/feed/',
                'https://www.super-funny.com/feed/',
                'http://slay.me/feed',
                'https://www.funny-jokes-quotes-sayings.com/funny-jokes.xml',
                'http://modest-jokes.blogspot.com/feeds/posts/default?alt=rss',
                'https://badkidsjokes.tumblr.com/rss',
                'http://chillyjokes.com/chillyjokes/jokes/feed/',
                'https://laughbreak.com/pictures/its-okay-to-feed-the-ducks-bread-now/',
                'https://acornyjokeaday.tumblr.com/rss',
                'https://somejokeshere.blogspot.com/feeds/posts/default?alt=rss'
            ]

            def parse(self, response, depth=0):
                storage[str(response.url)] = response.body.decode("utf-8")
                
        runner = crawler.CrawlerRunner()
        deferred = runner.crawl(JokeSpider)
        deferred.addBoth(lambda _: reactor.stop())
        reactor.run()
        return storage
    except Exception as e:
        return {'exception' : str(e)}

## Rap texts parsing

In [9]:
def read_genius_token():
    with open('genius_token.txt', 'r') as f:
        token = f.readline().strip()
    return token

genius = Genius(read_genius_token())

In [10]:
def get_rap_data():
    storage = {}
    try:
        class RapSpider(scrapy.Spider):
            name = "rap_checker"
            start_urls = [
                'https://bestlifeonline.com/funniest-rap-lyrics/'
            ]

            def parse(self, response, depth=0):
                resp = response.xpath("//div[@class='content noskimwords']").xpath("//h2/text()")
                for item in resp:
                    match = re.match("[0-9]+\. ([A-Za-z0-9\s\-\'\.]+), \"([A-Za-z0-9\s\-\'\.]+)\"", item.get().strip())
                    if match:
                        try:
                            author = match.group(1)
                            song = match.group(2)
                            name = author+'|'+song
                            print((author, song))
                            song = genius.search_song(song, author)
    #                         print(song.lyrics)
                            print(len(song.lyrics))
                            storage[name] = song.lyrics
                        except Exception as e:
                            print(f'!!! Exception passed on {name}', str(e))
                
        runner = crawler.CrawlerRunner()
        deferred = runner.crawl(RapSpider)
        deferred.addBoth(lambda _: reactor.stop())
        reactor.run()
        return storage
    except Exception as e:
        return {'exception' : str(e)}

In [101]:
storage_file = 'storage.pkl'
def is_rawdata_exist(storage_file):
    return os.path.exists(storage_file)
    # return False

if is_rawdata_exist(storage_file):
    print('LOADING')
    with open(storage_file, 'rb') as f:
        storage = pickle.load(f)
else:
    print('FROM WEB')
    storage = {}  
    storage['usagov'] = get_web_data(get_usagov_rss_data)
    storage['joke'] = get_web_data(get_joke_rss_data)
    storage['rap'] = get_web_data(get_rap_data)
    with open(storage_file, 'wb') as f:
        pickle.dump(storage, f)


LOADING


In [24]:
len(storage['joke'].keys())

17

In [25]:
storage.keys()

dict_keys(['usagov', 'joke', 'rap'])

In [26]:
len(storage['usagov'].keys())

27

In [27]:
list(storage['usagov'].values())[0]

'<?xml version="1.0" encoding="UTF-8"?><rss version="2.0"\n\txmlns:content="http://purl.org/rss/1.0/modules/content/"\n\txmlns:wfw="http://wellformedweb.org/CommentAPI/"\n\txmlns:dc="http://purl.org/dc/elements/1.1/"\n\txmlns:atom="http://www.w3.org/2005/Atom"\n\txmlns:sy="http://purl.org/rss/1.0/modules/syndication/"\n\txmlns:slash="http://purl.org/rss/1.0/modules/slash/"\n\t>\n\n<channel>\n\t<title>Africa &#8211; United States Department of State</title>\n\t<atom:link href="https://www.state.gov/rss-feed/africa/feed/" rel="self" type="application/rss+xml" />\n\t<link>https://www.state.gov</link>\n\t<description></description>\n\t<lastBuildDate>Mon, 25 Apr 2022 15:01:24 +0000</lastBuildDate>\n\t<language>en-US</language>\n\t<sy:updatePeriod>\n\thourly\t</sy:updatePeriod>\n\t<sy:updateFrequency>\n\t1\t</sy:updateFrequency>\n\t<generator>https://wordpress.org/?v=5.9.2</generator>\n\n<image>\n\t<url>https://www.state.gov/wp-content/uploads/2022/04/cropped-dos_seal-32x32.png</url>\n\t<tit

In [28]:
len(storage['joke'].keys())

17

In [29]:
list(storage['joke'].values())[0]

'<?xml version="1.0" encoding="UTF-8"?><rss version="2.0"\n\txmlns:content="http://purl.org/rss/1.0/modules/content/"\n\txmlns:wfw="http://wellformedweb.org/CommentAPI/"\n\txmlns:dc="http://purl.org/dc/elements/1.1/"\n\txmlns:atom="http://www.w3.org/2005/Atom"\n\txmlns:sy="http://purl.org/rss/1.0/modules/syndication/"\n\txmlns:slash="http://purl.org/rss/1.0/modules/slash/"\n\t xmlns:media="http://search.yahoo.com/mrss/" >\n\n<channel>\n\t<title>LaffGaff</title>\n\t<atom:link href="https://laffgaff.com/feed/" rel="self" type="application/rss+xml" />\n\t<link>https://laffgaff.com</link>\n\t<description>Really funny short jokes, puns, trivia, quotes and more!</description>\n\t<lastBuildDate>Thu, 14 Apr 2022 12:41:33 +0000</lastBuildDate>\n\t<language>en-US</language>\n\t<sy:updatePeriod>\n\thourly\t</sy:updatePeriod>\n\t<sy:updateFrequency>\n\t1\t</sy:updateFrequency>\n\t<generator>https://wordpress.org/?v=5.9.3</generator>\n\n<image>\n\t<url>https://laffgaff.com/wp-content/uploads/2018/0

In [30]:
len(storage['rap'].keys())

30

In [31]:
list(storage['rap'].values())[0]

"The World Lyrics[Verse 1: Casey Veggies]\nNice denim, nice women\nYeah she know that I’m quite winnin\nThis gon come at the right time, I’m ridin' round and I’m tryna get it\nFirst whip was a Audi coupe\nDrive that shit like a Honda Civic\nOne life, I’m tryna live it\nThat Hennessy is in my liver\nUsually I don’t drink but when I do it’s so exquisite\nI would probably be a bigger rapper if I would do, something different\nBut it’s no hype, it’s on sight\nI’d rather rap about my life\nNo judging, girl do yo thing\nWhy you worried bout what I think?\nI be rockin' them ripped up jeans\nI be movin' round with my team\nIt’s good to have different options, got different kicks from different shopping\nBound to get it, that ain’t no option\nWe gotta do it, put it all in music\nPeas and Carrots, yea we gon stay\nThat’s my brand like Frito Lay\n[Verse 2]\nWorried bout all the places we go, vacation in Rio\nI be in my speedos, everything wavy\nLike a torpedo, baby girl date me\nThen she gotta le

In [88]:
storage['rap']

{'Game|Dreams': "The World Lyrics[Verse 1: Casey Veggies]\nNice denim, nice women\nYeah she know that I’m quite winnin\nThis gon come at the right time, I’m ridin' round and I’m tryna get it\nFirst whip was a Audi coupe\nDrive that shit like a Honda Civic\nOne life, I’m tryna live it\nThat Hennessy is in my liver\nUsually I don’t drink but when I do it’s so exquisite\nI would probably be a bigger rapper if I would do, something different\nBut it’s no hype, it’s on sight\nI’d rather rap about my life\nNo judging, girl do yo thing\nWhy you worried bout what I think?\nI be rockin' them ripped up jeans\nI be movin' round with my team\nIt’s good to have different options, got different kicks from different shopping\nBound to get it, that ain’t no option\nWe gotta do it, put it all in music\nPeas and Carrots, yea we gon stay\nThat’s my brand like Frito Lay\n[Verse 2]\nWorried bout all the places we go, vacation in Rio\nI be in my speedos, everything wavy\nLike a torpedo, baby girl date me\nT

# Preprocess sentences for HuggingFace

In [116]:
storage_to_process = copy.deepcopy(storage)

map_labels = {'usagov' : 0, 'joke' : 1, 'rap' : 2}

def remove_html_tags(text):
    html_detector = re.compile(r'<.*?>')
    return html_detector.sub(r'', text)

def remove_links(text):
    links_remover = re.compile(r'https?://\S+|www\.\S+')
    return links_remover.sub(r'', text)

def preprocess_html(body, local_processed_data, topic_key):
#     l = 0
    soup = BeautifulSoup(body)
    p_tags = soup.find_all('p')
    for each in p_tags:
        processed = sent_tokenize(each.text.encode('ascii', 'ignore').decode('ascii').lower().strip())
        for processed_item in processed:
            processed_item = re.sub("[\d\.]+", '', processed_item).strip()
            if len(processed_item) > 0:
                if topic_key == 'joke':
                    if processed_item.startswith('a: ') or processed_item.startswith('q: '):
                        local_processed_data.append(processed_item[3:])
                    else:
                        local_processed_data.append(processed_item)
                elif topic_key == 'usagov':
                    if len(processed_item.split(' ')) > 3:
                        local_processed_data.append(processed_item)
                else:
                    raise Exception(f'Unknown topic_key for preprocess_html(): {topic_key}')
#             l += len(processed)
#     print(l)

def get_data_rss(substorage, topic_key):
    local_processed_data = []
    for link, body in substorage.items():
        preprocess_html(body, local_processed_data, topic_key)
    local_processed_labels = [map_labels[topic_key]] * len(local_processed_data)
    with open(f'tmp_{topic_key}.json', 'w') as f:
        json.dump({'section' : local_processed_data}, f, indent=4, sort_keys=True)
    return local_processed_data, local_processed_labels

def join_rap_sentences(local_lines, lines_in_group):
    rap_sentences = []
    idx = 0
    while idx < len(local_lines):
        line = local_lines[idx].strip().lower()
        if len(line) == 0:
            del local_lines[idx]
            continue
        if idx + lines_in_group > len(local_lines):
            rap_sentences.append(line)
        else:
            second_line = local_lines[idx + 1].strip().lower()
            if len(second_line) == 0:
                del local_lines[idx + 1]
                continue
            rap_sentences.append(line + ' ' + second_line)
        idx += lines_in_group
    return rap_sentences

# TODO how to have a deal with slang?
def get_approx_sentences_from_lyrics(lyrics, local_processed_data):
    couplets = re.split('\[.*\]' ,lyrics)
    for couplet in couplets:
        local_lines = couplet.split('\n')
        
        if len(local_lines) == 1:
            continue
        lines_in_group = 2
        rap_sentences = join_rap_sentences(local_lines, lines_in_group)
        local_processed_data.extend(rap_sentences)

def get_data_lyrics(substorage, topic_key):
    local_processed_data = []
    for lyrics in substorage.values():
        approx_sentences = get_approx_sentences_from_lyrics(lyrics, local_processed_data)
    local_processed_labels = [map_labels[topic_key]] * len(local_processed_data)
    with open(f'tmp_{topic_key}.json', 'w') as f:
        json.dump({'section' : local_processed_data}, f, indent=4, sort_keys=True)
    return local_processed_data, local_processed_labels

processed_data = []
processed_labels = []
for topic_key in map_labels.keys():
    if topic_key == 'joke' or topic_key == 'usagov':
        local_processed_data, local_processed_labels = get_data_rss(storage_to_process[topic_key], topic_key)
        processed_data.extend(local_processed_data)
        processed_labels.extend(local_processed_labels)
    elif topic_key == 'rap':
        local_processed_data, local_processed_labels = get_data_lyrics(storage_to_process[topic_key], topic_key)
        processed_data.extend(local_processed_data)
        processed_labels.extend(local_processed_labels)
    else:
        raise Exception(f'Unknown topic_key: {topic_key}')

    

/home/masdevas/.anaconda3/envs/nlp-unn/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [117]:
df_text_data = pd.DataFrame({'text' : processed_data, 'label' : processed_labels})

In [118]:
df_text_data

,text,label
0,"antony j blinken, secretary of state",0
1,"on behalf of the american people, i would like...",0
2,the united states and tanzania enjoy a longsta...,0
3,as tanzania also will soon celebrate years of...,0
4,i send my best wishes to the tanzanian people ...,0
...,...,...
23591,"i arrived in front of the dormitory ""yo, could...",2
23592,they showed me where it was for the moment i d...,2
23593,"so, i came to her room and opened the door oh,...",2
23594,a fella tongue-kissin' my girl in her mouth i ...,2


In [119]:
df_text_data['label'].value_counts()

0    20920
2     1375
1     1301
Name: label, dtype: int64

## Reduce number of samples of the largest class

In [141]:
index_of_usagov = df_text_data[df_text_data['label'] == 0].index
index_of_usagov
remove_n = 18000
drop_indices = np.random.choice(index_of_usagov, remove_n, replace=False)
df_text_data_red = df_text_data.drop(drop_indices).reset_index()

In [142]:
df_text_data_red['label'].value_counts()

0    2920
2    1375
1    1301
Name: label, dtype: int64

# HuggingFace transformers is coming!

## Prepare dataset

In [146]:
hf_dataset = Dataset.from_pandas(df_text_data_red).remove_columns('index')

In [147]:
hf_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 5596
})

In [148]:
hf_dataset[1254]

{'text': 'antony j blinken, secretary of state', 'label': 0}